In [0]:
'''
KEPBINCLASS----

This script predicts the morphological class of the Kepler binary star 
systems through deep learning algorithm and recognizing their light 
curve images.

Change the path of train and validation data directory values based your 
directory scheme. 

The script works on Python 3.x. Some modifications are needed to get it 
run in Python 2.x

The script uses TensorFlow 1.x. Some modifications are needed to get it 
run in TensorFlow 2.x
'''

# import necessary modules
import numpy as np
import matplotlib.pyplot as plt
%tensorflow_version 1.x
import tensorflow as tf
import os
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import regularizers
from keras import optimizers
from google.colab import files
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

# pass deprecated warnings from TF 2.0
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


img_width, img_height = 640, 480 # size of images

# train and validation directory
train_data_dir = 'drive/My Drive/newbin/newdata2/train'
validation_data_dir = 'drive/My Drive/newbin/newdata2/validation'
n_train = 1458 # number of train images
n_valid = 294 # number of validation images
epoch = 50 # number of epochs
batch = 16 # batch size

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


# the model
model = Sequential()
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001),
                 input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch,
    class_mode='categorical')


#fitting

history = model.fit_generator(
    train_generator,
    steps_per_epoch=n_train // batch,
    epochs=epoch,
    validation_data=validation_generator,
    validation_steps=n_valid // batch, shuffle=False)


#saving model

model.save_weights('mod_binclass.h5')


# plotting and saving the accuracy and loss function

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('acc.png')
plt.show()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('loss.png')
plt.show()


# Prediction - a specific file

img_width, img_height = 640, 480
img = image.load_img('drive/My Drive/newbin/newdata2/pred/co1.png', 
                     target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

pred = model.predict_classes(img) # class prediction
#pred = model.predict_proba(img) # probability prediction
print("---Prediction-----")
print(pred)


# Prediction - a bunch of images in folder

pred_data_dir = 'drive/My Drive/newbin/newdata2/pred'

images = []
for img in os.listdir(pred_data_dir):
    imgn = img
    img = os.path.join(pred_data_dir, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    classp = model.predict_classes(img)
    print(imgn,classp) 

Found 1458 images belonging to 3 classes.
Found 294 images belonging to 3 classes.
Epoch 1/50
91/91 [==============================] - 89s 983ms/step - loss: 0.7950 - acc: 0.8365 - val_loss: 0.4352 - val_acc: 0.8750
Epoch 2/50
91/91 [==============================] - 89s 975ms/step - loss: 0.2596 - acc: 0.9225 - val_loss: 0.1671 - val_acc: 0.9532
Epoch 3/50
91/91 [==============================] - 88s 967ms/step - loss: 0.2003 - acc: 0.9430 - val_loss: 0.2869 - val_acc: 0.9065
Epoch 4/50
91/91 [==============================] - 87s 956ms/step - loss: 0.1839 - acc: 0.9533 - val_loss: 0.1551 - val_acc: 0.9532
Epoch 5/50
91/91 [==============================] - 87s 954ms/step - loss: 0.1632 - acc: 0.9526 - val_loss: 0.1453 - val_acc: 0.9568
Epoch 6/50
91/91 [==============================] - 89s 979ms/step - loss: 0.1492 - acc: 0.9588 - val_loss: 0.1602 - val_acc: 0.9568
Epoch 7/50
91/91 [==============================] - 92s 1s/step - loss: 0.1587 - acc: 0.9663 - val_loss: 0.1402 - val_a

In [0]:
tf.__version__

'1.15.0'